In [1]:
import numpy as np

In [17]:
desacelerar = 0
acelerar = 1

acoes = [desacelerar, acelerar]

facil = 0
dificil = 1
exaustao = 2

estados = [facil, dificil, exaustao]

t = {}

t[desacelerar] = np.array([[1, 0, 0],
                           [0.7, 0.3, 0],
                           [0, 0, 1]])

t[acelerar] = np.array([[0.2, 0.8, 0],
                       [0, 0.7, 0.3],
                       [0, 0, 1]])

r = [1, 10, -100]

s0 = facil

gama = 0.9

In [18]:
q = {}

def obter_q(q, s, a):
  if (s,a) in q:
    return q[(s,a)]
  return 0

def atualizar_q(q, s, a, valor):
  q[(s, a)] = valor

def melhor_q_acao(q, s):
  melhor_q = -float('inf')
  melhor_acao = acoes[0]
  for a in acoes:
    qsa = obter_q(q,s,a)
    if qsa > melhor_q:
      melhor_q = qsa
      melhor_acao = a
  return melhor_q, melhor_acao

In [19]:
def proximo_estado(s,a):
  probs = t[a]
  s_novo = np.random.choice(estados, p=probs[s, :])
  return s_novo

In [20]:
max_episodios = 2000
max_passos = 100

epsilon = 0.5

alfa = 0.01

politica = [0 for estado in estados]

for episodio in range(max_episodios):

  s = s0

  for passo in range(max_passos):
    rec = r[s]
    rr = np.random.random()

    if rr < epsilon:

      a = np.random.choice(acoes)
      
    else:

      _, melhor_a = melhor_q_acao(q, s)
      a = melhor_a

    s_novo = proximo_estado(s,a)
    melhor_q_novo_estado, _ = melhor_q_acao(q,s_novo)
    alvo = rec + gama*melhor_q_novo_estado
    
    qsa = obter_q(q, s,a)
    novo_qsa = qsa + alfa*(alvo - qsa)

    atualizar_q(q, s, a, novo_qsa)

    s = s_novo


In [21]:
print(q)

{(0, 0): 50.28955246147485, (0, 1): 54.90604824667045, (1, 0): 61.00546159773877, (1, 1): -200.58219487928693, (2, 0): -999.9999999999429, (2, 1): -999.9999999999429}


## Interpretação

Ao variar os valores de epsilon ou alfa praticamente não ocorre alterações. Apenas temos variações nas recompensas de estando em 1, ficar em 1.
Contudo ao variar o valor de gama, ele tende a ficar em 1 se estiver em 1.

Minimizando epsilon conseguimos minimizar cair em um local minínmo, pois com o valor e epsilon menor exploramos mais outras possibilidades, buscando um global mínimo.
Variando o valor de alfa, variamos o tamnaho do passo, ou seja valores de alfa maiores damos passos maiores, mais tendemos a passar o mínimo e ter dificuldade de atingir, o ideial é um valor nem muito elevado e nem muito baixo.
Por fim sobre o impacto do valor de gama, se tivermos um valor elevado de gama não somos tão "gulosos" e gostariamos de explorar possibilidades, contudo dado um valor baixo de gama, queremos recompensas a curto prazo.